<h1>Title</h1>

<h1>Introduction</h1>

<p>Placeholder text</p>

<h1>Methods and Results</h1>

<p>Describe Methods here</p>
<p><b>MAKE SURE TO LABEL VISUALS (ex. Figure 1) and refer back to them in explanations</b></p>

In [2]:
# Load the data set + libraries: 
library(tidyverse)
library(repr)
library(tidymodels)
library(caret)

players_data <- read_csv("https://drive.google.com/uc?export=download&id=1Mw9vW0hjTJwRWx0bDXiSpYsO3gKogaPz")

sessions_data <- read_csv("https://drive.google.com/uc?export=download&id=14O91N5OlVkvdGxXNJUj5jIsV5RexhzbB")

Rows: 196 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, age
lgl (3): subscribe, individualId, organizationName

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


<p>PlaceHolder</p>
<p></p>

In [6]:
# Split the data into training and testing data + wrangling
players_data_split <- initial_split(players_data, prop=0.75, strata=played_hours)
players_data_train <- training(players_data_split)
players_data_test <- testing(players_data_split)

# wrangle to tidy

players_data_train

experience,subscribe,hashedEmail,played_hours,name,gender,age,individualId,organizationName
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<lgl>,<lgl>
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17,NA,NA
Regular,TRUE,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e,0.1,Kylie,Male,21,NA,NA
Regular,TRUE,8e594b8953193b26f498db95a508b03c6fe1c24bb5251d392c18a0da9a722807,0.0,Luna,Female,19,NA,NA
Amateur,FALSE,1d2371d8a35c8831034b25bda8764539ab7db0f63938696917c447128a2540dd,0.0,Emerson,Male,21,NA,NA
Amateur,TRUE,8b71f4d66a38389b7528bb38ba6eb71157733df7d1740371852a797ae97d82d1,0.1,Natalie,Male,17,NA,NA
Pro,TRUE,4caa42e1b20511552434978171dcf7283fb6eb857eb871327e39bfc328002309,0.0,Daniela,Male,17,NA,NA
Veteran,TRUE,6f9acf8ea9956fe817895c78d10e1e25c11aba335a451e5b0322b073ee5008c4,0.0,Niamh,Non-binary,17,NA,NA
Amateur,FALSE,2313a06afe47eacc28ff55adf6f072e7d12b0d12d7cbae29656f9029ec72bb53,0.0,Quinlan,Male,22,NA,NA
Beginner,TRUE,9db37a1d381697f6c83e3b97b276e38bb9d9c811ba4596edb75d4dbfea6415b0,0.0,Elodie,Female,17,NA,NA


<p>PlaceHolder</p>


<h1>asdasdasd</h1>
<p>asdasdsad</p>
<ul>
    <li>as</li>
    <li>ds</li>
</ul>

In [ ]:
#Code for Age

<p>PlaceHOlder
</p>

In [ ]:
#Code for Experience

<p>PlaceHolder</p>

In [ ]:
# Code for Regression

<p>PlaceHOlder</p>

<h1>Discussion</h1>

<p>PlaceHolder
</p>

<h1>References</h1>

<p>APA format??????</p>